In [1]:
import itertools # 是 Python 的內建模組，提供了一組用於處理迭代器的函數和工具。
                 # 它包含了各種用於高效處理迭代器的函數，可以幫助我們編寫更簡潔、高效的代碼。
import sys # 是 Python 的內建模組，提供了與 Python 解釋器和運行環境相關的功能。
# sys.path 是一個列表，包含了 Python 解釋器在導入模組時會搜尋的路徑。
# 當你使用 import 語句導入模組時 Python 會依次在 sys.path 中的路徑下尋找對應的模組文件。
sys.path.append("../ecg-classification/")
# sys.path.append("C:\\Users\\Chen_Lab01\\Documents\\GitHub/ecg-classification")
# from IPython.display import Video
# import cv2
from matplotlib import pyplot as plt
plt.style.use("ggplot") #  是 Matplotlib 庫中用於設置繪圖樣式的函數。它使用了一種名為 "ggplot" 的預定義樣式
import matplotlib
matplotlib.use("TkAgg")
                        #  該樣式模仿了 R 語言的 ggplot2 繪圖包的外觀。
# print(sys.path)
from ecg_tools.config import EcgConfig, Mode
from ecg_tools.data_loader import DatasetConfig, get_data_loaders
from ecg_tools.model import ECGformer
from ecg_tools.train import ECGClassifierTrainer


Load model

In [7]:
import torch
config = EcgConfig()    
model_quantized = torch.load("..\\model_save\\model_quantized.pth")
model = torch.load("..\\model_save\\model.pth")
model.eval()

ECGformer(
  (encoder): ModuleList(
    (0-5): 6 x TransformerEncoderLayer(
      (0): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MultiHeadAttention(
            (queries_projection): Linear(in_features=192, out_features=192, bias=True)
            (values_projection): Linear(in_features=192, out_features=192, bias=True)
            (keys_projection): Linear(in_features=192, out_features=192, bias=True)
            (final_projection): Linear(in_features=192, out_features=192, bias=True)
          )
          (2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): ResidualAdd(
        (block): Sequential(
          (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (1): MLP(
            (0): Linear(in_features=192, out_features=768, bias=True)
            (1): GELU(approximate='none')
            (2): Linear(in_features=768, out_features=192, bias=True)
          )
          (2):

In [10]:
import einops
loader = get_data_loaders(DatasetConfig())
accuracy = 0
for signal, label in loader[Mode.eval]:
    p = model(einops.rearrange(signal, "b c e -> b e c"))
    print(p)
    print(label)
    # print(signal.shape, label.shape)
    print(p.argmax(1) == label)
    accuracy += torch.sum(p.argmax(1) == label)
    print(f"accuracy: {accuracy / config.dataset.batch_size}")
    break



tensor([[ 1.2383e-02,  1.0049e+00, -1.5993e-02, -2.0682e-01, -8.0101e-01,
          2.0851e-01],
        [ 2.8313e-03,  9.5658e-01, -1.4441e-02, -2.1946e-01, -7.4981e-01,
          1.8808e-01],
        [ 6.2331e-03,  9.6735e-01, -1.6609e-02, -2.1761e-01, -7.5707e-01,
          1.9157e-01],
        [-2.6533e-02,  7.3442e-01, -3.0799e-02, -2.7630e-01, -5.0446e-01,
          9.5112e-02],
        [ 1.4834e-02,  1.0208e+00, -1.4969e-02, -2.0293e-01, -8.1724e-01,
          2.1497e-01],
        [ 1.4871e-02,  1.0261e+00, -1.3187e-02, -2.0148e-01, -8.2400e-01,
          2.1688e-01],
        [ 5.9424e-03,  9.6071e-01, -1.7472e-02, -2.1981e-01, -7.5220e-01,
          1.8982e-01],
        [ 1.1497e-03,  9.5339e-01, -1.1214e-02, -2.1834e-01, -7.4809e-01,
          1.8593e-01],
        [ 1.3534e-02,  1.0142e+00, -1.4435e-02, -2.0550e-01, -8.0950e-01,
          2.1122e-01],
        [ 1.0725e-03,  9.4053e-01, -1.6962e-02, -2.2477e-01, -7.3013e-01,
          1.8052e-01],
        [ 1.6763e-02,  1.0230e

## Parameter 提取

In [4]:
for name in model_quantized.state_dict():
   print(name)

print(model_quantized.state_dict()['encoder.0.0.block.1.queries_projection._packed_params._packed_params'])

positional_encoding
encoder.0.0.block.0.weight
encoder.0.0.block.0.bias
encoder.0.0.block.1.queries_projection.scale
encoder.0.0.block.1.queries_projection.zero_point
encoder.0.0.block.1.queries_projection._packed_params.dtype
encoder.0.0.block.1.queries_projection._packed_params._packed_params
encoder.0.0.block.1.values_projection.scale
encoder.0.0.block.1.values_projection.zero_point
encoder.0.0.block.1.values_projection._packed_params.dtype
encoder.0.0.block.1.values_projection._packed_params._packed_params
encoder.0.0.block.1.keys_projection.scale
encoder.0.0.block.1.keys_projection.zero_point
encoder.0.0.block.1.keys_projection._packed_params.dtype
encoder.0.0.block.1.keys_projection._packed_params._packed_params
encoder.0.0.block.1.final_projection.scale
encoder.0.0.block.1.final_projection.zero_point
encoder.0.0.block.1.final_projection._packed_params.dtype
encoder.0.0.block.1.final_projection._packed_params._packed_params
encoder.0.1.block.0.weight
encoder.0.1.block.0.bias
enco